In [ ]:
## Training code for custom evaluation metrics

import argparse
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score

def custom_eval(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds > 0.5 # You can adjust this threshold as needed
    precision = precision_score(labels, preds, average=None)
    recall = recall_score(labels, preds, average=None)
    # return a pair metric_name, result. The metric name must not contain a colon (:) or a space
    # since preds are margin(before logistic transformation, cutoff at 0)
    return [('precision_0', precision[0]), ('precision_1', precision[1]), ('recall_0', recall[0]), ('recall_1', recall[1])]

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Add arguments for data locations
    parser.add_argument('--train', type=str)
    parser.add_argument('--validation', type=str)

    # Add arguments for hyperparameters
    parser.add_argument('--eval_metric', type=str)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--rate_drop', type=float)
    parser.add_argument('--tweedie_variance_power', type=float)
    parser.add_argument('--seed', type=int)

    args = parser.parse_args()

    # Load your data from the Pipe mode input
    dtrain = xgb.DMatrix('/opt/ml/input/data/train')
    dval = xgb.DMatrix('/opt/ml/input/data/validation')

    # Set your parameters
    params = {
        'eval_metric': args.eval_metric,
        'objective': args.objective,
        'num_round': args.num_round,
        'rate_drop': args.rate_drop,
        'tweedie_variance_power': args.tweedie_variance_power,
        'seed': args.seed,
    }

    # Train your model using xgb.train and your custom evaluation metric
    bst = xgb.train(params, dtrain, num_round=args.num_round, feval=my_custom_eval_metric)

    # Save your model
    bst.save_model(os.path.join(args.sm_model_dir, 'xgboost-model'))